In [0]:
-- Exploratory Data Analysis: Incoming Radiology Reports Findings
-- ================================================================

-- 1. Dataset Overview & Basic Statistics
SELECT 
  COUNT(*) as total_reports,
  COUNT(DISTINCT patient_id) as unique_patients,
  COUNT(DISTINCT referring_physician) as unique_physicians,
  MIN(ingestion_date) as earliest_report,
  MAX(ingestion_date) as latest_report,
  DATEDIFF(MAX(ingestion_date), MIN(ingestion_date)) as date_range_days
FROM users.guy_livni.incoming_radiology_reports_findings;

-- 2. Temporal Analysis: Reports by Date
SELECT 
  DATE(ingestion_date) as report_date,
  COUNT(*) as report_count,
  SUM(CASE WHEN ground_truth_critical = true THEN 1 ELSE 0 END) as critical_count,
  ROUND(AVG(CASE WHEN ground_truth_critical = true THEN 1.0 ELSE 0.0 END) * 100, 2) as critical_percentage
FROM users.guy_livni.incoming_radiology_reports_findings
GROUP BY DATE(ingestion_date)
ORDER BY report_date DESC
LIMIT 30;

-- 3. Triage Category Distribution
SELECT 
  triage_category,
  COUNT(*) as count,
  ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage,
  SUM(CASE WHEN ground_truth_critical = true THEN 1 ELSE 0 END) as critical_cases
FROM users.guy_livni.incoming_radiology_reports_findings
GROUP BY triage_category
ORDER BY count DESC;

-- 4. Primary Findings Analysis
SELECT 
  ai_structured_data.primary_finding,
  COUNT(*) as occurrence_count,
  SUM(CASE WHEN ground_truth_critical = true THEN 1 ELSE 0 END) as critical_count,
  ROUND(AVG(CASE WHEN ground_truth_critical = true THEN 1.0 ELSE 0.0 END) * 100, 2) as critical_rate
FROM users.guy_livni.incoming_radiology_reports_findings
WHERE ai_structured_data.primary_finding IS NOT NULL
GROUP BY ai_structured_data.primary_finding
ORDER BY occurrence_count DESC
LIMIT 20;

-- 5. Affected Organ Distribution
SELECT 
  ai_structured_data.affected_organ,
  COUNT(*) as count,
  ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage,
  SUM(CASE WHEN ground_truth_critical = true THEN 1 ELSE 0 END) as critical_cases
FROM users.guy_livni.incoming_radiology_reports_findings
WHERE ai_structured_data.affected_organ IS NOT NULL
GROUP BY ai_structured_data.affected_organ
ORDER BY count DESC;

-- 6. Data Quality Assessment
SELECT 
  'Total Records' as metric,
  COUNT(*) as value
FROM users.guy_livni.incoming_radiology_reports_findings
UNION ALL
SELECT 'Missing Patient ID', COUNT(*) FROM users.guy_livni.incoming_radiology_reports_findings WHERE patient_id IS NULL
UNION ALL
SELECT 'Missing Patient Name', COUNT(*) FROM users.guy_livni.incoming_radiology_reports_findings WHERE patient_name IS NULL
UNION ALL
SELECT 'Missing Raw Text', COUNT(*) FROM users.guy_livni.incoming_radiology_reports_findings WHERE raw_text IS NULL
UNION ALL
SELECT 'Missing AI Structured Data', COUNT(*) FROM users.guy_livni.incoming_radiology_reports_findings WHERE ai_structured_data IS NULL
UNION ALL
SELECT 'Missing Triage Category', COUNT(*) FROM users.guy_livni.incoming_radiology_reports_findings WHERE triage_category IS NULL
UNION ALL
SELECT 'Missing Ground Truth', COUNT(*) FROM users.guy_livni.incoming_radiology_reports_findings WHERE ground_truth_critical IS NULL;

-- 7. Top Referring Physicians
SELECT 
  referring_physician,
  COUNT(*) as total_reports,
  SUM(CASE WHEN ground_truth_critical = true THEN 1 ELSE 0 END) as critical_reports,
  ROUND(AVG(CASE WHEN ground_truth_critical = true THEN 1.0 ELSE 0.0 END) * 100, 2) as critical_rate
FROM users.guy_livni.incoming_radiology_reports_findings
GROUP BY referring_physician
ORDER BY total_reports DESC
LIMIT 15;

-- 8. Contrast Usage Analysis
SELECT 
  ai_structured_data.is_contrast_used,
  COUNT(*) as count,
  ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage
FROM users.guy_livni.incoming_radiology_reports_findings
WHERE ai_structured_data.is_contrast_used IS NOT NULL
GROUP BY ai_structured_data.is_contrast_used
ORDER BY count DESC;